<a href="https://colab.research.google.com/github/superaja/learnings/blob/master/fastai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install fastai==0.7.0
!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python
import cv2
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
import torch
!pip install Pillow==4.0.0
!pip install image
%matplotlib inline
from fastai.imports import *

In [1]:
!pip install fastai==0.7.0
!pip3 install torchvision
!pip3 install torchtext==0.2.3

    100% |████████████████████████████████| 122kB 3.9MB/s 
    100% |████████████████████████████████| 3.6MB 1.3MB/s 
    100% |████████████████████████████████| 1.5MB 14.2MB/s 
    100% |████████████████████████████████| 184kB 23.9MB/s 
    100% |████████████████████████████████| 71kB 23.0MB/s 
    100% |████████████████████████████████| 61kB 23.6MB/s 
    100% |████████████████████████████████| 496.4MB 30kB/s 
    100% |████████████████████████████████| 61kB 19.8MB/s 
    100% |████████████████████████████████| 11.6MB 3.4MB/s 
    100% |████████████████████████████████| 92kB 22.3MB/s 
    100% |████████████████████████████████| 81kB 23.8MB/s 
  Running setup.py bdist_wheel for feather-format ... - done
  Stored in directory: /root/.cache/pip/wheels/85/7d/12/2dfa5c0195f921ac935f5e8f27deada74972edc0ae9988a9c1
  Running setup.py bdist_wheel for bcolz ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - 

In [0]:
from fastai.imports import *

In [0]:
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *